### Bag of word
> https://docs.google.com/presentation/d/1iLjWK9AQszJUSHxqDetBn5PXJ8SsmETcvCTNU2EGblE/edit#slide=id.p

目的: 處理不同長度的句子，但能表示字出現的頻率

#### Bag of word到TF-IDF
> https://taweihuang.hpd.io/2017/03/01/tfidf/

In [1]:
import pandas as pd
import numpy as np

Accept = pd.read_excel('ICLR_accepted.xlsx')
Reject = pd.read_excel('ICLR_rejected.xlsx')
Accept.head()

,Unnamed: 0,0
0,0,Minimal-Entropy Correlation Alignment for Unsu...
1,1,Large Scale Optimal Transport and Mapping Esti...
2,2,TRUNCATED HORIZON POLICY SEARCH: COMBINING REI...
3,3,Model-Ensemble Trust-Region Policy Optimization
4,4,A Neural Representation of Sketch Drawings


In [22]:
Accept.iloc[50:][0].head()

50    Unsupervised Representation Learning by Predic...
51    Emergent Communication in a Multi-Modal, Multi...
52    FastGCN: Fast Learning with Graph Convolutiona...
53    Emergent Translation in Multi-Agent Communication
54    An efficient framework for learning sentence r...
Name: 0, dtype: object

0是reject

In [2]:
X_train = pd.concat([Accept.iloc[50:][0], Reject.iloc[50:][0]]).reset_index(drop=True)
df1 = pd.DataFrame(np.ones(532))
df2 = pd.DataFrame(np.zeros(703))
y_train = pd.concat([df1, df2]).reset_index(drop=True)

X_test = pd.concat([Accept.iloc[:50][0], Reject.iloc[:50][0]]).reset_index(drop=True)
df3 = pd.DataFrame(np.ones(50))
df4 = pd.DataFrame(np.zeros(50))
y_test = pd.concat([df3, df4]).reset_index(drop=True)

In [5]:
# 參數設定
max_len = 30

In [ ]:
def shuffle(X,Y):
  np.random.seed(10)
  randomList = np.arange(X.shape[0])
  np.random.shuffle(randomList)
  return X[randomList], Y[randomList]

建立字典/ 手刻字典

In [86]:
# 大寫轉小寫
X_train = X_train.str.lower()
X_test = X_test.str.lower()
# 建立字典
set = [] # 紀錄有哪些不同的單字
sentence_dict = {}

for index, row in X_train.iteritems():
    # 從句子切割單字出來
    for word in row.split(' '):
        if word not in set:
            set.append(word)
            if sorted(sentence_dict.keys()):
                sentence_dict[sorted(sentence_dict.keys())[-1]+1] = word
            else:
                sentence_dict[1] = word

# 手動轉換句子
for index, row in X_train.iteritems():
    num_sentence = []
    for word in row.split(' '):
        # 找到該單字在字典裡的 id
        num_sentence.append(list(sentence_dict.keys())[list(sentence_dict.values()).index(word)])
    X_train[index] = num_sentence
    
for index, row in X_test.iteritems():
    num_sentence = []
    for word in row.split(' '):
        try:
            num_sentence.append(list(sentence_dict.keys())[list(sentence_dict.values()).index(word)])
        except:
            # 字不存在就忽略
            continue
    X_test[index] = num_sentence

In [87]:
# 自動補齊長度
x_train_seq = sequence.pad_sequences(X_train, maxlen=max_len)
x_test_seq  = sequence.pad_sequences(X_test,  maxlen=max_len)

In [75]:
 X_train.head()

0                      [0, 1, 2, 3, 4, 5, 6]
1              [7, 8, 9, 10, 11, 12, 13, 14]
2    [15, 16, 2, 17, 18, 19, 20, 21, 22, 23]
3                          [7, 24, 9, 25, 8]
4                [26, 27, 28, 29, 2, 30, 31]
Name: 0, dtype: object

In [71]:
# 取得某個單字的dictnary id
list(sentence_dict.keys())[list(sentence_dict.values()).index('scale')]

82

In [60]:
sentence_dict = {1: 'haha', 0: 'XD'}
# 排列後找最後一個
sorted(sentence_dict.keys())[-1]

1

In [52]:
X_train.head().str.split(' ')

0    [unsupervised, representation, learning, by, p...
1    [emergent, communication, in, a, multi-modal,,...
2    [fastgcn:, fast, learning, with, graph, convol...
3    [emergent, translation, in, multi-agent, commu...
4    [an, efficient, framework, for, learning, sent...
Name: 0, dtype: object

**底下這個cell是自動建立字典**

In [3]:
from keras.preprocessing.text import Tokenizer
# 自動轉小寫
token = Tokenizer(lower=True)
token.fit_on_texts(X_train)
# 根據字典將句子轉換成數字
X_train_seq = token.texts_to_sequences(X_train)
X_test_seq = token.texts_to_sequences(X_test)

統一長度

In [6]:
from keras.preprocessing import sequence
x_train_seq = sequence.pad_sequences(X_train_seq, maxlen=max_len)
x_test_seq  = sequence.pad_sequences(X_test_seq,  maxlen=max_len)

In [34]:
len(token.word_index)

2137

### model

In [7]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers import CuDNNLSTM
from keras.layers.embeddings import Embedding

model_LSTM = Sequential()

# Embedding shape (字典詞的數量/ batch_size, 每個字的長度, 輸出的維度)
model_LSTM.add(Embedding(output_dim=5, input_dim=len(token.word_index), input_length=max_len)) # 沒有 input_length也能訓練
# model_LSTM.add(Embedding(output_dim=44, input_dim=len(sentence_dict), input_length=max_len))
# LSTM輸入(samples，timesteps，input_dim），輸出(samples，timesteps，output_dim）
model_LSTM.add(CuDNNLSTM(30, return_sequences=True))
# model_LSTM.add(CuDNNLSTM(50, return_sequences=True))
# model_LSTM.add(CuDNNLSTM(40, return_sequences=True))
# model_LSTM.add(CuDNNLSTM(30, return_sequences=True))
# model_LSTM.add(CuDNNLSTM(20, return_sequences=True))
model_LSTM.add(CuDNNLSTM(20, return_sequences=False))

model_LSTM.add(Dropout(0.25))

# model_LSTM.add(Dense(20, activation='relu'))
model_LSTM.add(Dense(10, activation='relu'))
model_LSTM.add(Dense(2, activation='softmax'))

model_LSTM.summary()

model_LSTM.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

W0705 23:14:54.317387 13484 deprecation_wrapper.py:119] From c:\users\wade\.virtualenvs\19'_summer_vacation-ib8vnh7u\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0705 23:14:54.329567 13484 deprecation_wrapper.py:119] From c:\users\wade\.virtualenvs\19'_summer_vacation-ib8vnh7u\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0705 23:14:54.330569 13484 deprecation_wrapper.py:119] From c:\users\wade\.virtualenvs\19'_summer_vacation-ib8vnh7u\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0705 23:14:55.150772 13484 deprecation_wrapper.py:119] From c:\users\wade\.virtualenvs\19'_summer_vacation-ib8vnh7u\lib\site-packages\keras\backend\tensorflow_backend.py:133: The name tf.placeholder_with_defa

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 5)           10685     
_________________________________________________________________
cu_dnnlstm_1 (CuDNNLSTM)     (None, None, 30)          4440      
_________________________________________________________________
cu_dnnlstm_2 (CuDNNLSTM)     (None, 20)                4160      
_________________________________________________________________
dropout_1 (Dropout)          (None, 20)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                210       
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 22        
Total params: 19,517
Trainable params: 19,517
Non-trainable params: 0
_________________________________________________________________


In [8]:
model_LSTM.fit(x_train_seq, y_train,batch_size=20, epochs=10,verbose=2, validation_split=0.2, shuffle=True)

W0705 23:14:57.098093 13484 deprecation.py:323] From c:\users\wade\.virtualenvs\19'_summer_vacation-ib8vnh7u\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 988 samples, validate on 247 samples
Epoch 1/10
 - 3s - loss: 0.6933 - acc: 0.5395 - val_loss: 0.7351 - val_acc: 0.0000e+00
Epoch 2/10
 - 0s - loss: 0.6923 - acc: 0.5476 - val_loss: 0.7419 - val_acc: 0.0000e+00
Epoch 3/10
 - 0s - loss: 0.6897 - acc: 0.5385 - val_loss: 0.7671 - val_acc: 0.0000e+00
Epoch 4/10
 - 0s - loss: 0.6861 - acc: 0.5405 - val_loss: 0.9175 - val_acc: 0.0000e+00
Epoch 5/10
 - 0s - loss: 0.6737 - acc: 0.5526 - val_loss: 0.7544 - val_acc: 0.2105
Epoch 6/10
 - 0s - loss: 0.5393 - acc: 0.7561 - val_loss: 1.9330 - val_acc: 0.2510
Epoch 7/10
 - 0s - loss: 0.3414 - acc: 0.8654 - val_loss: 1.4879 - val_acc: 0.3644
Epoch 8/10
 - 0s - loss: 0.2143 - acc: 0.9211 - val_loss: 1.9180 - val_acc: 0.3684
Epoch 9/10
 - 0s - loss: 0.1515 - acc: 0.9575 - val_loss: 2.0282 - val_acc: 0.4696
Epoch 10/10
 - 0s - loss: 0.1077 - acc: 0.9646 - val_loss: 2.6147 - val_acc: 0.4575


### test model

In [92]:
model_LSTM.evaluate(x_test_seq, y_test)

100/100 [==============================] - 0s 463us/step


[2.6161777496337892, 0.65]

#### train的validation達到60趴, test上反而比較低

In [95]:
model_LSTM.evaluate(x_test_seq, y_test)

100/100 [==============================] - 0s 165us/step


[3.4139516735076905, 0.57]